# Generate synthetic data by GPT4 and fine tune GPT3.5（GPT4による合成データを作成し、GPT3.5をfine tuning）

---





Result (Accuracy)
- fine tuned model : 0.7
- base model : 0.53


## 1 Generate synthetic data（合成データを作成します）

### 1-1 import library（ライブラリをインポートします）

In [ ]:
!pip -q install langchain openai tiktoken  python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
import time
from dotenv import load_dotenv
from langchain import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from langchain import PromptTemplate, LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI

from langchain.schema.output_parser import StrOutputParser

In [ ]:
import time
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, classification_report

create an explanation about the model（モデルについての説明を作成します）

In [ ]:
model = "A model that creates a mail of customer complaints from customers who are dissatisfied. Select randomly only one of these prodcucts to talk about in this mail. 'Mortgage', 'Checking or savings account', 'Student loan', 'Money transfer, virtual currency, or money service', 'Bank account or service', 'Consumer Loan'.  The email must provide a detailed description of the product's features. This mail of complaints should be in 500 words"
temperature=.9
number_of_examples = 15

### 1-2. Generate synthetic data （合成データを作成します）

In [ ]:
import os
import openai
import random

openai.api_key = "*****" #input your API key

def generate_example(model, prev_examples, temperature=temperature):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n\nptoduct\n-----------\n$product_goes_here\n-----------\n```\n\nOnly one prompt/response/ptoduct pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{model}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 7:
            prev_examples = random.sample(prev_examples, 2)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })
    print("messages:", messages)

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        temperature=temperature,
        max_tokens=1354,
    )

    print(response.choices[0])
    return response.choices[0].message['content']

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(model, prev_examples, temperature)
    prev_examples.append(example)

#print(prev_examples)

### 1-3. Create dataframe（dataframeを生成）

In [ ]:
import pandas as pd

prompts = []
responses = []
products = []

for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
    products.append(split_example[5].strip())
  except:
    pass

df = pd.DataFrame({
    'prompt': prompts,
    'response': responses,
    'products': products
})

df = df.drop_duplicates()

pd.set_option("display.max_colwidth", 5000)

In [ ]:
df

,prompt,response,products
0,A customer has been facing issues with their Mortgage service. They are confused about the terms and they believe the interest rates are too high. Create a 500-word email of complaint.,"Subject: Complaint Regarding Mortgage Service\n\nDear Sir/Madam,\n\nI hope this message finds you in good health. I am writing this letter to draw your attention to a few issues I have been facing with the mortgage service provided by your organization.\n\nHaving been a loyal customer of your bank for several years, I have always appreciated your commitment to customer satisfaction. However, my recent experiences have left me disappointed and confused. The terms associated with my mortgage loan are quite complex and lack clear explanations. While I understand that mortgage documents are typically filled with jargon, I expected a more customer-friendly approach from your team to help me understand these terms better. This lack of clarity has created a barrier to managing my loan effectively.\n\nAdditionally, I have noticed that the interest rates on my mortgage loan are significantly higher than the current market rates. This has made me question the fairness and transparency of your services. I have browsed other financial institutions' offerings and noticed that their rates are more competitive. This discrepancy has not only added financial strain but has also led me to question the integrity of your institution.\n\nI understand that interest rates may vary depending on various factors, but I was not expecting such a vast difference. I believe that customers should be treated fairly and should be made aware of any changes that directly impact them. I, unfortunately, feel blindsided by the high interest rates and the lack of communication from your side.\n\nIn light of these issues, I request your immediate attention in resolving my concerns. Please provide me with an easy-to-understand breakdown of my mortgage terms and lend clarity on why the interest rates applied to my loan are so high.\n\nI look forward to a resolution that upholds the values of transparency, fairness, and customer satisfaction. Your swift response and corrective measures will be highly appreciated.\n\nThank you for your understanding and cooperation.\n\nYours Truly,\n\n[Customer's Name]",Mortgage
1,A customer is dissatisfied with their Checking or savings account service. They are dissatisfied due to unexpected fees and poor customer service. Create a 500-word email of complaint.,"Subject: Urgent Complaint About Checking or Savings Account Service \n\nDear Sir/Madam,\n\nI trust this message receives your urgent attention. I am writing to address my dissatisfaction with your Checking or Savings Account services.\n\nAs a long-standing patron of your bank, I have always been impressed by the quality of your financial products and services. Sadly, the events of recent months have created a sense of disillusionment and disappointment in me that I feel compelled to express.\n\nFirstly, I am alarmed at the number of unexpected charges that have been levied on my accounts lately. These charges, which were not explicitly outlined when I opened the account, have posed significant inconveniences as they impact my financial planning. This arbitrary application of fees is a cause of major concern for me, and I believe this to be a lapse in the commitment to transparency that your organization is known for.\n\nSecondly, I am saddened by the quality of customer service I've received in relation to these issues. I reached out to your customer service department on multiple occasions to seek clarity about these unfamiliar charges, only to be met with disappointing responses. The lack of promptness, empathy, and comprehensive assistance from your team has exacerbated my problems and created a sense of mistrust in my mind.\n\nFor these reasons, I would like to request your immediate intervention in this matter. I urge you to review the charges applied to my accou

In [ ]:
df.to_csv('synthetic_data15_20230916v2.csv')

### 1-4. Create training data as json （jsonフォーマットで訓練データ作成）

In [ ]:
train_df=df

train_df.to_json('train.jsonl15_20230916v2', orient='records', lines=True)
#test_df.to_json('test.jsonl10_20230916', orient='records', lines=True)

## 2. Load data（dataをインポート）


In [ ]:
X=pd.read_csv("CC_20k_20230114.csv")
X

,Product,Issue,text,Complaint ID,Ja
0,Checking or savings account,Managing an account,XX/XX/2022. I began noticing charges to my debit card and changes to my online Wells Fargo account .I have been issues 4 debit cards and now Wells Fargo refuses to issue a new debit card and suspended my online service .My arm pin keeps getting changed and I can not stop the fraud charges.,6301154,xx/xx/2022。私は自分のデビットカードへの料金とオンラインウェルズファーゴアカウントの変更に気づき始めました。私は4つのデビットカードに問題があり、ウェルズファーゴは新しいデビットカードの発行を拒否し、オンラインサービスを停止しました。詐欺料金を止めないでください。
1,Checking or savings account,Managing an account,"I have a joint account with my Daughter. She goes to XXXX in XXXX XXXX, and I live and work in XXXX XXXX. There was a Fraudulent ATM withdrawal in Florida on XX/XX/XXXX for {$1000.00}. The account only had around {$240.00} balance at the time of withdrawal. Wells Fargo ATM allowed the transaction because they said the account had overdraft protection. There had never been a legitimate {$1000.00} withdrawal from that account and neither I nor my daughter ever asked for overdraft protection and not sure how it was added. I immediately contacted Wells Fargo and reported it as fraud and filed a claim. They said they'll investigate it and on XX/XX/XXXX responded that they denied the claim and said I must have provided my account info to someone else to withdraw on my behalf. That is a lie. This was a fraudulent transaction that Wells Fargo allowed and they need to accept responsibility for it.",6280688,私は娘と共同アカウントを持っています。彼女はxxxx xxxxでxxxxに行き、私はxxxx xxxxに住んで働いています。{$ 1000.00}のxx/xx/xxxxでフロリダで詐欺的なATM撤退がありました。アカウントは、撤退時に{$ 240.00}の残高しかありませんでした。Wells Fargo ATMは、アカウントに当座貸越保護があると言ったため、取引を許可しました。そのアカウントからの正当な{$ 1000.00}の撤退は一度もなかったし、私も娘も当座貸越保護を求めたことはなく、それがどのように追加されたかわからない。私はすぐにウェルズ・ファーゴに連絡し、それを詐欺として報告し、請求を提出しました。彼らはそれを調査すると言い、xx/xx/xxxxで、彼らは主張を否定したと答え、私に代わって撤退するために他の誰かにアカウント情報を提供したに違いないと言いました。それは嘘です。これは、ウェルズ・ファーゴが許可した詐欺的な取引であり、彼らはそれに対する責任を受け入れる必要があります。
2,Mortgage,Trouble during payment process,"I made a principal payment of {$9900.00} on XX/XX/22. US Bank withdrew {$9900.00} from my XXXX XXXX checking account. However, US Bank has not given me credit for the principal payment. US Bank shows on my account activity a payment of {$9900.00} and then a payment reversal of {$9900.00}. When I spoke with customer service manager, I was assured that the {$9900.00} would be returned to my XXXX XXXX checking account. When after nearly a week I did not receive the {$9900.00}, I called back US Bank. I am now told by the customer service representative that the {$9900.00} seems to be lost in the process and was told an investigation has been requested. The investigation will take about XXXX business days. I want the {$9900.00} returned to me immediately and I do not want to be charged the interest on my XXXX loan for this amount while my money is missing due to the fault of US Bank.",6257306,xx/xx/22で{$ 9900.00}の主要な支払いを行いました。米国銀行は、XXXX XXXX当座預金口座から{$ 9900.00}を撤回しました。しかし、米国銀行は私に元本の支払いに対する功績を与えていません。米国の銀行は、私のアカウント活動に{$ 9900.00}の支払いを示し、{$ 9900.00}の支払いの逆転を示しています。カスタマーサービスマネージャーと話をしたとき、{$ 9900.00}がXXXX XXXX当座預金口座に返品されると確信していました。ほぼ1週間後、{$ 9900.00}を受け取らなかったとき、私は米国銀行を呼びました。私は現在、カスタマーサービス担当者から、{$ 9900.00}がその過程で失われたようで、調査が要求されたと言われたと言われています。調査には約xxxx営業日がかかります。私は{$ 9900.00}をすぐに返したいと思っていますが、米国銀行の過失のために私のお金が欠けている間、この金額に対してXXXXローンの利息を請求したくありません。
3,Checking or savings account,Managing an account,"I created a Savings account XX/XX/2022 with Ally Bank, and submitted a deposit of {$400.00}, which was immediately withdrawn from my depositing account. At the time of the creation of the account, I was able to register for Online access, but was unable to access the online platform. I have been continuously unable to login to access my account or funds. On XX/XX/XXXX, I tried messaging customer support on their Website, and received no response. On XX/XX/XXXX, I called the company and asked for help, to which they told me a support ticket would be opened, and Id be contacted shortly. It never happened. Ive been 

In [ ]:
Xp=X[0:5000]
Xp["Product"].value_counts()

Checking or savings account                           2000
Mortgage                                              1660
Student loan                                           628
Money transfer, virtual currency, or money service     428
Bank account or service                                192
Consumer Loan                                           92
Name: Product, dtype: int64

In [ ]:
X=X[5000:]
X["Product"].value_counts()

Mortgage                                              88101
Checking or savings account                           52372
Student loan                                          26787
Money transfer, virtual currency, or money service    13184
Bank account or service                                9546
Consumer Loan                                          4405
Name: Product, dtype: int64

In [ ]:
Xtext=X[["text"]]
Xtext

,text
5000,"In XXXX we applied for a line of credit with a credit union to consolidate some debts. Their search turned up XXXX line of credit loans with PNC bankXXXX XXXX XXXX XXXX old line that I thought had expired years ago. The other XXXX XXXX ) was a current line that was an active account. I called PNC customer service and was told to FAX a letter to remove the loans from the books. I decided to do these XXXX XXXX XXXX ) at a time so to avoid confusion. After hearing nothing more about the loan for two weeks, the PNC customer service person said it would take about ten days, I called again. The PNC customer service person said they never got my FAX and to resend the letter again which I did. About a week later I went to a local PNC branch office to talk to some XXXX in person about the issues I was having. While there I spoke with another PNC customer service person and she told me that the loan office had never received my second FAX. I sent another request to remove this loan from there and went home. When I got home the mail was in and with it was a letter from PNC that they had removed ( XXXX ) from the books. I was confused but happy, XXXX down XXXX more to go XXXX XXXX ). On XXXX I sent PNC the early termination fee and a message to clear and remove from the books line of credit ... XXXX. PNC 's Response was to FAX or mail a letter to them with this request so I mailed the request certified mail. In two weeks I called PNC again and was informed that they had not received my request. When I told them that I had a receipt stamped by their receiver the customer service person put me on hold and transferred me to another customer service person. This transferring me around took me through XXXX other operators. With the third XXXX I ended the call and told the operator that I was tired of being transferred around and would seek higher authority 's. \nI feel that this bank is trying to force me to stay with them by making getting a loan at another bank so difficult. And that I just give up and get the loan at PNC."
5001,XX/XX/XXXX M & T bank called me and stated my account was hacked for {$2000.00}. I took the rest of my money out and M & T staff told me to file a police report. I did file the report and never heard back from the bank or the police.
5002,On XX/XX/XXXX I purchased a watch online and paid {$4500.00} to the seller using XXXX. The seller has since stopped all contact and refuses to reply to my messages or answer calls even though I know they are receiving them. I called Chase to help and they closed the claim right away and passed the information to XXXX.
5003,"Title is exactly as stated. \n\nTD Bank back in XXXX closed my account because of delinquency. I had to pay off that amount before opening a new one with a balance of {$790.00}. The amount was paid in person, cash, in XXXX. Now in XXXX as of XX/XX/XXXX I was charged {$460.00} for an offset as well as on XX/XX/XXXX charged {$520.00} for a total of {$970.00}. I was told there's also a remaining balance of {$270.00}. \n\nI have contacted every single person I can manage to get ahold of. The boss of one particular store with a job title of "" Business Analysts Supervisor/Account supervisor '' contacted me and told me that I could dispute the claim but there was no way they can reverse the offset. Also last week when I was in person they said they would do the exact same thing and nobody called me to give me an answer. I called back and spoke with the boss with a fancy long title. So I then called customer service. Since it was an offset they were unable to dispute it and said they would connect me to someone who could. I was connected with their Recovery department. The agent was not able to dispute the claim either. The only thing she noticed and was able to do was inquire and fill out an investigation of why I was being charged over the amount of the balance owed, which still doesn't solve the original problem of being charged 2x for something that was 

In [ ]:
ylabel=X[["Product"]]
ylabel

,Product
5000,Consumer Loan
5001,Checking or savings account
5002,"Money transfer, virtual currency, or money service"
5003,Checking or savings account
5004,Mortgage
...,...
199390,Bank account or service
199391,Student loan
199392,Mortgage
199393,Consumer Loan


## 3. Prepare training and test data （訓練データと検証データを作成）


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(Xtext, ylabel, stratify=ylabel, train_size=0.052, test_size=0.0055, random_state=200)
len(X_train), len(y_train), len(X_valid), len(y_valid)

(10108, 10108, 1070, 1070)

### 3-1  Create test data (検証データを作成)

In [ ]:
X_valid

,text
87436,"PayPal asked me for a photo ID, I have used my PayPal account for 20 + years already. PayPal put a limited access tag on my account, I received ZERO NOTICE of this. PayPal is asking me for receipts of items from a distributor I sell on EBay but I do not have receipts as all these items were bought by me on XXXX over the past 20 years. I am simply unloading/selling a collection of rock patches that average about {$9.00} a piece. I do not operate a business on XXXX. I have perfect 100 % + positive feedback on XXXX too, so there should be no need for PayPal to limit my account. By limiting my account, PayPal will not allow me to withdraw that funds I have in my PayPal account. I tried calling PayPal but when you hear the automated system state there is a wait time of 1 hour to 2 hours and they are going to transfer you to put you in line to wait for a customer service agent, PayPal then drops the call or in other words PayPal hangs up on you. There is zero way to resolve the matter online or over the phone then."
69550,"I took out a private student loan with Citibank ( XXXX ) in XXXX to attend graduate school at XXXX. Upon completion, I began to pay back the loan ( in XXXX XXXX ). There were XXXX private loans total with XXXX. In XXXX XXXX, Citi sold XXXX of the loans to XXXX. I began to make payments to XXXX, however in late XXXX, due to financial problems, I was unable to make the ballooned payments. I eventually worked out a payment plan with XXXX in XXXX XXXX. We set up a repayment plan for both of my loans with XXXX. XXXX withdrew directly from my checking account to post towards the loan. However, in XXXX, while still on my repayment plan, Citi took XXXX of the loans back and I was not notified by either XXXX or Citi. During the past year ( from XXXX XXXX XXXX ), I was unaware that Citi had taken the loan back. XXXX continued to withdraw money from my account for the repayment plan. I was also still paying back a loan with Citi and I never saw the second loan on my loan account. It was n't until I ran a credit report on myself in XXXX XXXX that I noticed I had a "" Charge Off '' loan with Citi. When I contacted them to ask why, they said they took the loan back from XXXX due to delinquency ( even though I already had an agreement in place with XXXX ) to pay the loan. Even though Citi took the loan back, they never contacted me for an entire year regarding the loan. When I spoke to Citi 's Recovery Department, they informed me that they would send the loan to a collections agency to process. I asked if I could just arrange a payment plan with them now since I was never aware of the loan. They said no. In XXXX, I was contacted by a collection agency to begin collecting on my loan. I am in complete disbelief that neither XXXX nor Citi notified me of a loan that was in limbo for an entire year. Now, it has ruined my credit. I have submitted a letter requesting that Citi take back the loan from the collection agency and process it themselves and allow me the opportunity to make the payment directly to Citi. I have also requested that they reverse all the negative action on my credit report."
80330,I have been trying to have my home loan modified with help from XXXX. Each time they have called the bank to check on the modification they were advised that they did n't have authorization. I have given XXXX authorization which has been sent to the bank on multiple times. The bank advised they had received the authorization on XX/XX/XXXX and then on XX/XX/XXXX they said they did n't have. This has left me in a terrible hardship and I 'm may lose my home. I lost my wife on XX/XX/XXXX. She battled XXXX for five years. Not only have I had to endure watching my wife suffer and leave this world I may now have to leave our home that we shared together. My son and I live in the home and very much want to continue living here. The bank has advised that there is a sale date for my home on XX/XX/XXXX. My son and I have no where to 

In [ ]:
y_valid

,Product
87436,Checking or savings account
69550,Student loan
80330,Mortgage
152574,Bank account or service
62991,Checking or savings account
...,...
81388,Checking or savings account
175519,"Money transfer, virtual currency, or money service"
72446,Student loan
68202,Mortgage


In [ ]:
y_valid.nunique()

Product    6
dtype: int64

In [ ]:
data = pd.concat([y_valid, X_valid], axis=1)
data

,Product,text
87436,Checking or savings account,"PayPal asked me for a photo ID, I have used my PayPal account for 20 + years already. PayPal put a limited access tag on my account, I received ZERO NOTICE of this. PayPal is asking me for receipts of items from a distributor I sell on EBay but I do not have receipts as all these items were bought by me on XXXX over the past 20 years. I am simply unloading/selling a collection of rock patches that average about {$9.00} a piece. I do not operate a business on XXXX. I have perfect 100 % + positive feedback on XXXX too, so there should be no need for PayPal to limit my account. By limiting my account, PayPal will not allow me to withdraw that funds I have in my PayPal account. I tried calling PayPal but when you hear the automated system state there is a wait time of 1 hour to 2 hours and they are going to transfer you to put you in line to wait for a customer service agent, PayPal then drops the call or in other words PayPal hangs up on you. There is zero way to resolve the matter online or over the phone then."
69550,Student loan,"I took out a private student loan with Citibank ( XXXX ) in XXXX to attend graduate school at XXXX. Upon completion, I began to pay back the loan ( in XXXX XXXX ). There were XXXX private loans total with XXXX. In XXXX XXXX, Citi sold XXXX of the loans to XXXX. I began to make payments to XXXX, however in late XXXX, due to financial problems, I was unable to make the ballooned payments. I eventually worked out a payment plan with XXXX in XXXX XXXX. We set up a repayment plan for both of my loans with XXXX. XXXX withdrew directly from my checking account to post towards the loan. However, in XXXX, while still on my repayment plan, Citi took XXXX of the loans back and I was not notified by either XXXX or Citi. During the past year ( from XXXX XXXX XXXX ), I was unaware that Citi had taken the loan back. XXXX continued to withdraw money from my account for the repayment plan. I was also still paying back a loan with Citi and I never saw the second loan on my loan account. It was n't until I ran a credit report on myself in XXXX XXXX that I noticed I had a "" Charge Off '' loan with Citi. When I contacted them to ask why, they said they took the loan back from XXXX due to delinquency ( even though I already had an agreement in place with XXXX ) to pay the loan. Even though Citi took the loan back, they never contacted me for an entire year regarding the loan. When I spoke to Citi 's Recovery Department, they informed me that they would send the loan to a collections agency to process. I asked if I could just arrange a payment plan with them now since I was never aware of the loan. They said no. In XXXX, I was contacted by a collection agency to begin collecting on my loan. I am in complete disbelief that neither XXXX nor Citi notified me of a loan that was in limbo for an entire year. Now, it has ruined my credit. I have submitted a letter requesting that Citi take back the loan from the collection agency and process it themselves and allow me the opportunity to make the payment directly to Citi. I have also requested that they reverse all the negative action on my credit report."
80330,Mortgage,I have been trying to have my home loan modified with help from XXXX. Each time they have called the bank to check on the modification they were advised that they did n't have authorization. I have given XXXX authorization which has been sent to the bank on multiple times. The bank advised they had received the authorization on XX/XX/XXXX and then on XX/XX/XXXX they said they did n't have. This has left me in a terrible hardship and I 'm may lose my home. I lost my wife on XX/XX/XXXX. She battled XXXX for five years. Not only have I had to endure watching my wife suffer and leave this world I may now have to leave our home that we shared together. My son and I live in the home and very much want to continue living here. The bank has advised that there is a sale date

### 3.2 Prepare training data (訓練データの作成)

In [ ]:
sX=pd.read_csv("synthetic_data15_20230916v2.csv")
sX

,Unnamed: 0,prompt,response,products
0,0,A customer has been facing issues with their Mortgage service. They are confused about the terms and they believe the interest rates are too high. Create a 500-word email of complaint.,"Subject: Complaint Regarding Mortgage Service\n\nDear Sir/Madam,\n\nI hope this message finds you in good health. I am writing this letter to draw your attention to a few issues I have been facing with the mortgage service provided by your organization.\n\nHaving been a loyal customer of your bank for several years, I have always appreciated your commitment to customer satisfaction. However, my recent experiences have left me disappointed and confused. The terms associated with my mortgage loan are quite complex and lack clear explanations. While I understand that mortgage documents are typically filled with jargon, I expected a more customer-friendly approach from your team to help me understand these terms better. This lack of clarity has created a barrier to managing my loan effectively.\n\nAdditionally, I have noticed that the interest rates on my mortgage loan are significantly higher than the current market rates. This has made me question the fairness and transparency of your services. I have browsed other financial institutions' offerings and noticed that their rates are more competitive. This discrepancy has not only added financial strain but has also led me to question the integrity of your institution.\n\nI understand that interest rates may vary depending on various factors, but I was not expecting such a vast difference. I believe that customers should be treated fairly and should be made aware of any changes that directly impact them. I, unfortunately, feel blindsided by the high interest rates and the lack of communication from your side.\n\nIn light of these issues, I request your immediate attention in resolving my concerns. Please provide me with an easy-to-understand breakdown of my mortgage terms and lend clarity on why the interest rates applied to my loan are so high.\n\nI look forward to a resolution that upholds the values of transparency, fairness, and customer satisfaction. Your swift response and corrective measures will be highly appreciated.\n\nThank you for your understanding and cooperation.\n\nYours Truly,\n\n[Customer's Name]",Mortgage
1,1,A customer is dissatisfied with their Checking or savings account service. They are dissatisfied due to unexpected fees and poor customer service. Create a 500-word email of complaint.,"Subject: Urgent Complaint About Checking or Savings Account Service \n\nDear Sir/Madam,\n\nI trust this message receives your urgent attention. I am writing to address my dissatisfaction with your Checking or Savings Account services.\n\nAs a long-standing patron of your bank, I have always been impressed by the quality of your financial products and services. Sadly, the events of recent months have created a sense of disillusionment and disappointment in me that I feel compelled to express.\n\nFirstly, I am alarmed at the number of unexpected charges that have been levied on my accounts lately. These charges, which were not explicitly outlined when I opened the account, have posed significant inconveniences as they impact my financial planning. This arbitrary application of fees is a cause of major concern for me, and I believe this to be a lapse in the commitment to transparency that your organization is known for.\n\nSecondly, I am saddened by the quality of customer service I've received in relation to these issues. I reached out to your customer service department on multiple occasions to seek clarity about these unfamiliar charges, only to be met with disappointing responses. The lack of promptness, empathy, and comprehensive assistance from your team has exacerbated my problems and created a sense of mistrust in my mind.\n\nFor these reasons, I would like to request your immediate intervention in this matter. I urge you to review the charges appl

In [ ]:
X_train_sub=sX[["response"]]
X_train_sub

,response
0,"Subject: Complaint Regarding Mortgage Service\n\nDear Sir/Madam,\n\nI hope this message finds you in good health. I am writing this letter to draw your attention to a few issues I have been facing with the mortgage service provided by your organization.\n\nHaving been a loyal customer of your bank for several years, I have always appreciated your commitment to customer satisfaction. However, my recent experiences have left me disappointed and confused. The terms associated with my mortgage loan are quite complex and lack clear explanations. While I understand that mortgage documents are typically filled with jargon, I expected a more customer-friendly approach from your team to help me understand these terms better. This lack of clarity has created a barrier to managing my loan effectively.\n\nAdditionally, I have noticed that the interest rates on my mortgage loan are significantly higher than the current market rates. This has made me question the fairness and transparency of your services. I have browsed other financial institutions' offerings and noticed that their rates are more competitive. This discrepancy has not only added financial strain but has also led me to question the integrity of your institution.\n\nI understand that interest rates may vary depending on various factors, but I was not expecting such a vast difference. I believe that customers should be treated fairly and should be made aware of any changes that directly impact them. I, unfortunately, feel blindsided by the high interest rates and the lack of communication from your side.\n\nIn light of these issues, I request your immediate attention in resolving my concerns. Please provide me with an easy-to-understand breakdown of my mortgage terms and lend clarity on why the interest rates applied to my loan are so high.\n\nI look forward to a resolution that upholds the values of transparency, fairness, and customer satisfaction. Your swift response and corrective measures will be highly appreciated.\n\nThank you for your understanding and cooperation.\n\nYours Truly,\n\n[Customer's Name]"
1,"Subject: Urgent Complaint About Checking or Savings Account Service \n\nDear Sir/Madam,\n\nI trust this message receives your urgent attention. I am writing to address my dissatisfaction with your Checking or Savings Account services.\n\nAs a long-standing patron of your bank, I have always been impressed by the quality of your financial products and services. Sadly, the events of recent months have created a sense of disillusionment and disappointment in me that I feel compelled to express.\n\nFirstly, I am alarmed at the number of unexpected charges that have been levied on my accounts lately. These charges, which were not explicitly outlined when I opened the account, have posed significant inconveniences as they impact my financial planning. This arbitrary application of fees is a cause of major concern for me, and I believe this to be a lapse in the commitment to transparency that your organization is known for.\n\nSecondly, I am saddened by the quality of customer service I've received in relation to these issues. I reached out to your customer service department on multiple occasions to seek clarity about these unfamiliar charges, only to be met with disappointing responses. The lack of promptness, empathy, and comprehensive assistance from your team has exacerbated my problems and created a sense of mistrust in my mind.\n\nFor these reasons, I would like to request your immediate intervention in this matter. I urge you to review the charges applied to my account and provide an in-depth explanation for each one. Additionally, I implore you to examine the handling of my case by your customer service department and take steps to ensure such shortcomings in service are not repeated.\n\nMy continued relationship with your bank hinges on the satisfactory resolution of these issues. I earnestly hope you will take the necessary actions to rectify these pr

In [ ]:
y_train_sub = sX[["products"]]
y_train_sub

,products
0,Mortgage
1,Checking or savings account
2,Student loan
3,"Money transfer, virtual currency, or money service"
4,Bank account or service
5,Consumer Loan
6,Mortgage
7,Checking or savings account
8,Checking or savings account
9,Mortgage


### 3-3. Prepare json file （jsonファイルを作成）

In [ ]:
import json

list_message = []

for i in range(15):
    instruction = X_train_sub.iloc[i,0]
    output = y_train_sub.iloc[i,0]
    message = [
        {"role": "user", "content": instruction},
        {
            "role": "assistant",
            "content": output,
        },
    ]
    list_message.append(message)

with open("output_20230916v2.jsonl", "w") as file:
    for messages in list_message:
        json_line = json.dumps({"messages": messages})
        file.write(json_line + '\n')

In [ ]:
list_message[14]

[{'role': 'user',
  'content': 'Subject: Genuine Complaint Regarding Mortgage Service\n\nDear Sir/Madam,\n\nI am writing to express my sincere grievance with your mortgage services, which I have been a part of for the past several years.\n\nMy dissatisfaction emanates from two main issues—the recent hike in mortgage rates and the inadequate communication surrounding it. One of my initial attractions to your services was the affordable interest rates offered, which, regrettably, have seen a significant and unanticipated increase recently. This has put strain on my financial planning and has caused undue stress.\n\nIn addition, I am dismayed at the lack of communication about these rate changes. It was only after I noticed the change in my monthly repayments that I was informed about this rate increase. The sudden and unexpected change, coupled with the lack of communication, is disturbing and leads to a sense of distrust.\n\nCustomers put great faith in their mortgage providers, and any

### 3-4 Upload jsonl file（jsonlファイルをアップロード）

In [ ]:
upload_file = openai.File.create(
  file=open("output_20230916v2.jsonl", "rb"),
  purpose='fine-tune'
)

## 4. fine-tune ChatGPT-3.5 （ChatGPT-3.5をファインチューン）

In [ ]:
suffix_name = "S-CC_20230916v2"

response = openai.FineTuningJob.create(
    training_file=upload_file["id"],
    #validation_file=validation_file_id,
    model="gpt-3.5-turbo-0613",
    suffix=suffix_name,
)

job_id = response["id"]

print(response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-vFH2eGijnJYuHd90q4VqyOpI",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1694860750,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-qXH1LjiWXH8L4UJtDpKX7IuV",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-Hi5ZgSbXoGtcG253oWNG9v0j",
  "hyperparameters": {
    "n_epochs": 6
  },
  "trained_tokens": null,
  "error": null
}


In [ ]:
# List 10 fine-tuning jobs
status = openai.FineTuningJob.list(limit=10)
status["data"][0]["status"]

'succeeded'

In [ ]:
import os
import openai
import random

openai.api_key = "*****" #input your API key

## 5. Classify customer complaints by Fine-tuned model（Fine-tuned modelによりクレームを判別）

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "*****" #input your API key

In [ ]:
mt=1500

model = ChatOpenAI(temperature=0, model=status["data"][0]["fine_tuned_model"], max_tokens=mt)
prompt = ChatPromptTemplate.from_template("{foo}")
chain = prompt | model

chain.invoke({"foo": data.iloc[0,1]})

AIMessage(content='It seems that the person is frustrated with PayPal because they have been asked to provide a photo ID and receipts for items they are selling on eBay. They claim to have been using PayPal for over 20 years without any issues and have a perfect feedback score on eBay. They are unable to withdraw funds from their PayPal account due to the account limitation. The person tried contacting PayPal but was unable to reach customer service as the call was dropped after a long wait time. They feel that there is no way to resolve the issue online or over the phone.', additional_kwargs={}, example=False)

In [ ]:
%%time

# number of test samples
m=100
result=[]

for i in range(0,m):
  prt = f"""system : you can answer only one word.
            User: What products do people complaint of in the mail delimited by triple backticks? select one of these prodcucts.: 'Mortgage', 'Checking or savings account', 'Student loan', 'Money transfer, virtual currency, or money service', 'Bank account or service', 'Consumer Loan'
                '''{data.iloc[i,1]}'''
            Assistant:
            """
  cl=chain.invoke({"foo": prt}).content
  result.append(cl)

result[0:10]

CPU times: user 960 ms, sys: 103 ms, total: 1.06 s
Wall time: 1min 44s


['PayPal',
 'Student loan',
 'Mortgage',
 'Checking or savings account',
 'Checking or savings account',
 'Mortgage',
 'Bank account',
 'Consumer Loan',
 'Mortgage',
 'Checking or savings account']

In [ ]:
s=result.copy()
for i in range(0,m):
    if "Bank account or service" in s[i]:
      s[i]=0
    elif "Checking or savings account" in s[i]:
      s[i]=1
    elif "Consumer Loan" in s[i]:
      s[i]=2
    elif "Money transfer, virtual currency, or money service" in s[i]:
      s[i]=3
    elif "Mortgage" in s[i]:
      s[i]=4
    elif "Student loan" in s[i]:
      s[i]=5
    else:
      s[i]=6

s[0:10]

[6, 5, 4, 1, 1, 4, 6, 2, 4, 1]

In [ ]:
label=list(data['Product'])[0:m]

for i in range(0,m):
    if "Bank account or service" in label[i]:
      label[i]=0
    elif "Checking or savings account" in label[i]:
      label[i]=1
    elif "Consumer Loan" in label[i]:
      label[i]=2
    elif "Money transfer, virtual currency, or money service" in label[i]:
      label[i]=3
    elif "Mortgage" in label[i]:
      label[i]=4
    elif "Student loan" in label[i]:
      label[i]=5
    else:
      label[i]=6

label[0:10]

[1, 5, 4, 0, 1, 4, 3, 1, 4, 0]

## 6. Evaluate the fine-tuned model (fine-tuned modelの精度検証)

In [ ]:
print("result:", s)
print("label:", label)

result: [6, 5, 4, 1, 1, 4, 6, 2, 4, 1, 4, 0, 4, 3, 4, 6, 4, 5, 1, 4, 0, 6, 4, 0, 4, 4, 1, 0, 4, 0, 1, 2, 0, 4, 0, 5, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 0, 4, 0, 1, 4, 4, 1, 4, 0, 4, 4, 4, 5, 4, 5, 0, 1, 2, 4, 5, 0, 2, 4, 0, 0, 4, 5, 1, 5, 6, 6, 0, 4, 4, 2, 6, 4, 4, 1, 4, 4, 1, 2, 5, 4, 4, 4, 4, 4, 0, 0, 5, 4, 4]
label: [1, 5, 4, 0, 1, 4, 3, 1, 4, 0, 4, 1, 4, 3, 4, 5, 4, 5, 1, 4, 1, 0, 4, 3, 4, 4, 1, 1, 4, 1, 1, 3, 1, 4, 3, 5, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 1, 4, 1, 1, 4, 4, 1, 4, 1, 4, 4, 4, 5, 4, 5, 0, 1, 2, 4, 5, 1, 2, 4, 1, 0, 4, 4, 1, 5, 1, 3, 1, 4, 2, 2, 1, 4, 4, 1, 4, 4, 1, 3, 5, 4, 4, 4, 4, 4, 1, 1, 5, 4, 4]


In [ ]:
data['Product'].value_counts()

Mortgage                                              485
Checking or savings account                           288
Student loan                                          147
Money transfer, virtual currency, or money service     73
Bank account or service                                53
Consumer Loan                                          24
Name: Product, dtype: int64

In [ ]:
print(classification_report(label, s, digits=4))
print('accuracy score: {0:0.4f}'.format(accuracy_score(label, s)))


              precision    recall  f1-score   support

           0     0.1111    0.4000    0.1739         5
           1     0.8333    0.3571    0.5000        28
           2     0.5000    0.7500    0.6000         4
           3     1.0000    0.1429    0.2500         7
           4     0.9783    0.9783    0.9783        46
           5     0.9000    0.9000    0.9000        10
           6     0.0000    0.0000    0.0000         0

    accuracy                         0.7000       100
   macro avg     0.6175    0.5040    0.4860       100
weighted avg     0.8689    0.7000    0.7302       100

accuracy score: 0.7000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 7. Compare the fine-tuned model against base-gpt3.5 (base-gpt3.5と比較)

In [ ]:
mt=1500

model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", max_tokens=mt)
prompt = ChatPromptTemplate.from_template("{foo}")
chain = prompt | model

In [ ]:
%%time

# number of test samples
m=100
result=[]

for i in range(0,m):
  prt = f"""system : you can answer only one word.
            User: What products do people complaint of in the mail delimited by triple backticks? select one of these prodcucts.: 'Mortgage', 'Checking or savings account', 'Student loan', 'Money transfer, virtual currency, or money service', 'Bank account or service', 'Consumer Loan'
                '''{data.iloc[i,1]}'''
            Assistant:
            """
  cl=chain.invoke({"foo": prt}).content
  result.append(cl)

CPU times: user 4.04 s, sys: 534 ms, total: 4.57 s
Wall time: 10min 57s


In [ ]:
s=result.copy()
for i in range(0,m):
    if "Bank account or service" in s[i]:
      s[i]=0
    elif "Checking or savings account" in s[i]:
      s[i]=1
    elif "Consumer Loan" in s[i]:
      s[i]=2
    elif "Money transfer, virtual currency, or money service" in s[i]:
      s[i]=3
    elif "Mortgage" in s[i]:
      s[i]=4
    elif "Student loan" in s[i]:
      s[i]=5
    else:
      s[i]=6

In [ ]:
print(classification_report(label, s, digits=4))
print('accuracy score: {0:0.4f}'.format(accuracy_score(label, s)))


              precision    recall  f1-score   support

           0     0.1250    0.8000    0.2162         5
           1     0.0000    0.0000    0.0000        28
           2     0.5000    0.2500    0.3333         4
           3     0.0000    0.0000    0.0000         7
           4     0.9773    0.9348    0.9556        46
           5     1.0000    0.5000    0.6667        10
           6     0.0000    0.0000    0.0000         0

    accuracy                         0.5300       100
   macro avg     0.3718    0.3550    0.3103       100
weighted avg     0.5758    0.5300    0.5304       100

accuracy score: 0.5300


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

### notice
This code is solely for educational purpose. The code cannot be used for investments or busineeses in practice. TOSHI STATS Co.,Ltd. and I do not accept any responsibility or liability for loss or damage occasioned to any person or property through using materials, instructions, methods, algorithm or ideas contained herein, or acting or refraining from acting as a result of such use. TOSHI STATS Co.,Ltd. and I expressly disclaim all implied warranties, including merchantability or fitness for any particular purpose. There will be no duty on TOSHI STATS Co.,Ltd. and me to correct any errors or defects in the codes and the software.